<a href="https://colab.research.google.com/github/shivangisharma2103/Crime_visualization/blob/main/Neurons_TYL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing required packages and dependencies for the project
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
#unzipping both the test and train files for processing
!unzip '/content/train.csv.zip' -d '/content/train_data'
!unzip '/content/test.csv.zip' -d '/content/test_data'

Archive:  /content/train.csv.zip
  inflating: /content/train_data/train.csv  
Archive:  /content/test.csv.zip
  inflating: /content/test_data/test.csv  


In [ ]:
#reading the files as dataframe using pandas.
train_data = pd.read_csv('/content/train_data/train.csv')
pred_data = pd.read_csv('/content/test_data/test.csv')

In [ ]:
#The clean_data function is used to clean both the train and test data.
#The function takes in a dataframe and decides it's type based on it's shape.
#if the shape of the the dataframe[1] == 9 the it is a train data frame and processes according to it
#for test dataframe the processing is done accordingly.
def clean_data(data,shape):
  data['year'] = data['Dates'].apply(lambda x : x.split()[0].split('-')[0])
  data['Month'] = data['Dates'].apply(lambda x : x.split()[0].split('-')[1])
  data['Day'] = data['Dates'].apply(lambda x : x.split()[0].split('-')[2])
  time = data['Dates'].apply(lambda x : x.split()[1].split(':')[:2])
  data['time'] = [int(x[0])*60 + int(x[1]) for x in time]
  data.drop('Dates',axis = 1, inplace = True)
  data=pd.concat([data,pd.get_dummies(data['DayOfWeek'])],axis=1)
  data.drop('DayOfWeek',axis=1,inplace=True)
  data = pd.concat([data,pd.get_dummies(data['PdDistrict'])],axis = 1)
  data.drop('PdDistrict',axis = 1, inplace = True)
  if(shape==9):
    data.drop(['Descript','Resolution','Address'],axis = 1, inplace = True)
    X = data.drop('Category',axis = 1)
    y = pd.get_dummies(data['Category'])
    X = X.astype(float)
    return X,y
  else:
    data=data.drop(['Id'],axis=1)
    data.drop('Address',axis = 1, inplace = True)
    X=data.astype(float)
    return X


X,y=clean_data(train_data,train_data.shape[1]) #get the X and y values for the train data 
X_pred = clean_data(pred_data, pred_data.shape[1]) #get the X value for test data

#we split the train data into training and validation using the sklearn train-test split module.
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(790244, 23) (790244, 39)
(87805, 23) (87805, 39)


In [ ]:
#this function creates a tensorflow model
#the model consists of six dense layers with activation function as relu for each layers
#the last layer is a dense layer with 39 output neurons which has output as softmax, used for multiclass classification 
#the model uses categorical crossentropy as it's loss function and adam as an optimizer. The model evaluates itself on the accuracy metrics. 
#The categorical crossentropy when combined with accuracy metric are log_loss for the model
def my_model():

  model = tf.keras.models.Sequential([
                                      tf.keras.layers.Dense(100, input_shape=(X.shape[1],)),
                                      tf.keras.layers.Dense(80, activation='relu'),
                                      tf.keras.layers.Dense(64, activation='relu'),
                                      tf.keras.layers.Dense(64, activation='relu'),
                                      tf.keras.layers.Dense(64,activation='relu'),
                                      tf.keras.layers.Dense(64, activation='relu'),
                                      tf.keras.layers.Dense(39, activation='softmax')
  ])
  model.summary()

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = my_model()

model.fit(X_train, y_train, batch_size=64,epochs=10,verbose=2,validation_data=(X_test,y_test))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 100)               2400      
_________________________________________________________________
dense_45 (Dense)             (None, 80)                8080      
_________________________________________________________________
dense_46 (Dense)             (None, 64)                5184      
_________________________________________________________________
dense_47 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_48 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_49 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_50 (Dense)             (None, 39)               

In [ ]:
pred=model.predict(X_pred) #predicting with the model
pred_final=[np.argmax(i) for i in pred]
m_reshape = np.max(pred, axis=1).reshape(-1, 1)
pred_final = np.array((pred == m_reshape), dtype='int32')

In [ ]:
#A list of all categories of crimes present in the dataset.
sub_cols=list(sample_submission.columns)
sub_cols.remove('Id')
submission = pd.DataFrame({'Id':pred_data['Id']})
for i , entry in enumerate(s_sub_col):
    submission[entry] = pred_final[:,i]

submission.to_csv('/content/submission.csv', index=False)

40